In [2]:
# https://www.automationjames.com/2021/11/05/using-python-for-automation-with-simple-salesforce/
# https://developer.salesforce.com/blogs/2021/09/how-to-automate-data-extraction-from-salesforce-using-python 

from simple_salesforce import Salesforce
import requests
import pandas as pd
from io import StringIO

In [ ]:
sf = Salesforce(username='',password='', security_token='')


In [ ]:
# Option 1: download a Salesforce report
#Your Salesforce Instance URL
sf_instance = 'https://oneappexchange.lightning.force.com/' 
reportId = '' # add report id
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df1 = pd.read_csv(StringIO(download_report))

In [ ]:
# Option 2: query the data using SOQL
# Finding Metadata


descri=sf.UserInstall__c.describe()
[field['name'] for field in descri['fields']]



In [ ]:
# Fetching the data with SOQL


results=sf.query_all("""
    Select 
    CreatedDate,
    Listing__r.RecordTypeSnapshot__c,
    Name,
    Listing__r.ProviderName__c
    from UserInstall__c
    where CreatedDate=LAST_N_DAYS:7 
    """)

In [ ]:
#Constructing a DataFrame from a dictionary

records = [dict(CreatedDate=rec['CreatedDate'], 
                Record_Type=rec['Listing__r']['RecordTypeSnapshot__c'],
                Name=rec['Name'],
                ProviderName=rec['Listing__r']['ProviderName__c']) 
                for rec in results['records']]
df=pd.DataFrame(records)